In [ ]:
pip install yake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00


In [ ]:
pip install transformers

In [ ]:
pip install nltk

In [ ]:
import nltk

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 11.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [ ]:
pip install summarizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summarizer: filename=summarizer-0.0.7-py2.py3-none-any.whl size=284209 sha256=adb5d67b997cd9b866dcc7e8fa86077de5e208d4ca410c0642b7499ad7ada90f
  Stored in directory: /root/.cache/pip/wheels/20/bb/2d/1fe057c2f729818a5f28c312c3667e8b9d5cfd4af4a39895e7
Successfully built summarizer


In [ ]:
import yake # For Keyword Extraction
from transformers import BartTokenizer, BartForConditionalGeneration # For Text Summarization
#For aspect based sentimental analysis
import nltk
from nltk.corpus import stopwords
import stanza

In [ ]:
# For Keyword Extraction
def extract_keywords(text, language="en", max_keywords=10):
    # Create a YAKE keyword extractor
    kw_extractor = yake.KeywordExtractor(lan=language, n=max_keywords, dedupLim=0.7, dedupFunc='seqm')

    # Extract keywords
    keywords = kw_extractor.extract_keywords(text)

    # Deduplicate keywords using a set and combining similar keywords
    unique_keywords = set()
    deduplicated_keywords = []
    for keyword, score in keywords:
        # Check for similar keywords already in the set
        similar_keywords = [kw for kw in unique_keywords if kw in keyword or keyword in kw]

        if not similar_keywords:
            deduplicated_keywords.append(keyword)
            unique_keywords.add(keyword)

    # Return the list of deduplicated keywords
    return deduplicated_keywords


In [ ]:
#Text Summarization
# Loading pre-trained model and tokenizer

def text_summarization(product_review):
    model_name = "facebook/bart-large-cnn"
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    # Tokenizing and generating summary
    inputs = tokenizer.encode("summarize: " + product_review, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=50, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary



In [ ]:
# Here instead of narrowsing the sentiment of aspects into positive or negative,
#we are returning the sentiment word and leaving the choice to the user itself for judging the sentiment of the aspect.
#( Clearly shown in the output)
def aspect_sentiment_analysis(txt, stop_words, nlp):

    txt = txt.lower()  # Lowercasing the given Text
    sentList = nltk.sent_tokenize(txt)  # Splitting the text into sentences

    fcluster = []
    dic = {}

    for line in sentList:
        txt_list = nltk.word_tokenize(line)  # Splitting up into words
        taggedList = nltk.pos_tag(txt_list)  # Doing Part-of-Speech Tagging to each word

        doc = nlp(line)  # Object of Stanza NLP Pipeline

        # Getting the dependency relations between the words
        dep_node = []
        for dep_edge in doc.sentences[0].dependencies:
            dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])

        # Converting it into an appropriate format
        for i in range(0, len(dep_node)):
            if (int(dep_node[i][1]) != 0):
                dep_node[i][1] = txt_list[(int(dep_node[i][1]) - 1)]

        featureList = []
        for i in taggedList:
            if i[1].startswith('JJ'):  # Filter adjectives
                featureList.append(i[0])

        for i in featureList:
            filist = []
            for j in dep_node:
                if ((j[0] == i or j[1] == i) and j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"]):
                    if j[0] == i:
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i, filist])

    for i in fcluster:
        aspect = i[0]
        related_adjectives = ' '.join(i[1]).replace(' ', '')  # Combine words and remove spaces

        # Check for negation and adjust sentiment
        if "not" in i[1]:
            aspect_tokens = txt.split()
            not_index = i[1].index("not")
            if not_index < len(aspect_tokens) - 1:
                next_word = aspect_tokens[not_index + 1]
                aspect = "not_" + aspect if next_word in stop_words else "not " + aspect
            related_adjectives = related_adjectives.replace("not", "")

        if aspect not in dic:
            dic[aspect] = related_adjectives
        else:
            dic[aspect] += ' ' + related_adjectives

    finalcluster = [[value, [key]] for key, value in dic.items()]
    return finalcluster

nlp = stanza.Pipeline()
stop_words = set(stopwords.words('english'))



INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
# Can be softcoded so that user can enter their specific required review below given is sample review by our group
Product_Review = """
The new smartphone is fantastic. The camera quality is excellent, capturing sharp and clear photos.
The battery life exceeds expectations, lasting all day with regular use.
The sleek design and vibrant display make it a pleasure to use. However,it has high price.
"""


In [ ]:
#Original_Review
print("Original Review:\n", Product_Review)
summary = text_summarization(Product_Review)
#Summary
print("\nGenerated Summary:\n", summary)
# Extracting deduplicated keywords
keywords = extract_keywords(Product_Review)
print()
# Printing the keywords
print("Keywords:", keywords)
# Printing the aspects
print()
print(aspect_sentiment_analysis(Product_Review, stop_words, nlp))


Original Review:
 
The new smartphone is fantastic. The camera quality is excellent, capturing sharp and clear photos. 
The battery life exceeds expectations, lasting all day with regular use. 
The sleek design and vibrant display make it a pleasure to use. However,it has high price.



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]


Generated Summary:
 The camera quality is excellent, capturing sharp and clear photos. The battery life exceeds expectations, lasting all day with regular use. The sleek design and vibrant display make it a pleasure to use.

Keywords: ['smartphone is fantastic', 'capturing sharp and clear photos', 'camera quality is excellent', 'battery life exceeds expectations', 'lasting all day with regular', 'sleek design and vibrant display make it a pleasure', 'However,it has high price']

[['smartphone', ['new']], ['smartphone', ['fantastic']], ['quality', ['excellent']], ['photos', ['sharp']], ['', ['clear']], ['use', ['regular']], ['design', ['sleek']], ['display', ['vibrant']], ['price', ['high']]]
